### Sentence Embedding with Sentence BERT

In [1]:
# load the trained model from task 1
import torch

model_path = "artefacts/bert_mlm.pt"

trained_model = torch.load(model_path, map_location="cpu")

OSError: [WinError 1114] A dynamic link library (DLL) initialization routine failed. Error loading "d:\Anaconda\Lib\site-packages\torch\lib\c10.dll" or one of its dependencies.